In [1]:
# -*- coding: utf-8 -*-


# Importa librerías y define variables globales
import glob
import time
import locale
import calendar
from datetime import datetime
import decimal
import math
import pandas as pd
import calendar


#Ojo: es clave establecer que el lenguaje local es el español

from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.text.run import Font, Run
from docx.shared import Inches,Pt

locale.setlocale(locale.LC_ALL, 'esp_ESP')

dias_en_anno = 365.2425

In [2]:
# esta funcion mide el tiempo en años que ha pasado entre dos fechas. Trunca al entero más próximo.
def dif_fechas(fecha1, fecha2):
    a=abs(fecha2-fecha1).days
    b=math.trunc(float(a)/dias_en_anno)
    return str(b)

In [3]:
# esta función escribe la fecha en español
def texto_fecha(fecha):
    texto=str(fecha.day)+' de '+calendar.month_name[fecha.month]+' de '+str(fecha.year)
    return texto

In [4]:
# Acá se edita la ubicación donde están los datos de los futuros anexos
archivo_data_anexos='D:\\Mis Documentos\\12 Python Scripts\\Desarrollo Anexos\\anexos.xlsx'
data_anexos=pd.read_excel(archivo_data_anexos,sheetname='Hoja1')

In [14]:
hoy=datetime.utcnow()

# Llena los distintos anexos. Es importante codificar en utf-8 para asegurar que salga en español

for filas in range(len(data_anexos)):
    nombre=str(data_anexos['Nombres'].iloc[filas]+' '+data_anexos['Apellidos'].iloc[filas]).encode('utf-8')
    fecha_de_nacimiento=data_anexos['fechaNacimiento'].iloc[filas].to_pydatetime()
    RUT=str(data_anexos['RUT'].iloc[filas])
    estado_civil=str(data_anexos['estadoCivil'].iloc[filas]).encode('utf-8')
    nombreCalle=str(data_anexos['CalleDomicilio'].iloc[filas]).encode('utf-8')
    numeroCalle=str(data_anexos['numeroDomicilio'].iloc[filas]).encode('utf-8')
    nombreComuna=data_anexos['comunaDomicilio'].iloc[filas].encode('utf-8')
    fecha_de_contrato=data_anexos['fechaContrato'].iloc[filas].to_pydatetime()
    fecha_de_inicio_relacion=data_anexos['fechaEntradaEnVigencia'].iloc[filas].to_pydatetime()
    sueldo_base=328544
    bono_antiguedad=32768
    
    
    
    p1='En Santiago, a '+texto_fecha(hoy)+' entre la ASOCIACIÓN CHILENA DE SEGURIDAD, representada por don ALEXIS ILAN '\
    'FISCHMAN y denominada más adelante \"La Asociación\" o \"El Empleador\", ambos domiciliados en Ramón Carnicer 163, '\
    'comuna de Providencia de esta ciudad por una parte, y por la otra don(ña) '+ nombre+', de nacionalidad CHILENA , '\
    'fecha de nacimiento '+texto_fecha(fecha_de_nacimiento)+' , de '+ dif_fechas(fecha_de_nacimiento,hoy)+ ' años de '\
    'edad '+', Cédula Nacional de Identidad Nº '+RUT+ ', estado civil '+ estado_civil+', contratado(a) en esta ciudad, '\
    'con domicilio en calle '+nombreCalle+ ' N° '+numeroCalle+ ' comuna de '+nombreComuna+ ', denominada más adelante '\
    'el Trabajador(a), se ha convenido en la siguiente modificación del contrato individual de trabajo:\n'
    
    p2='PRIMERO: Con fecha '+texto_fecha(fecha_de_contrato)+' se suscribió un contrato individual de  trabajo entre el '\
    'Empleador y el Trabajador(a) '+nombre+' el cual comenzó a regir entre las partes con '\
    'fecha ' +texto_fecha(fecha_de_inicio_relacion)+'.\n'
    
    p3='SEGUNDA: Las partes vienen a sustituir el(los) Artículo(s) CUARTO del contrato mencionado en la cláusula primera de '\
    'este documento, por motivo de Nuevas condiciones contractuales, reemplazándose por el(los) siguiente(s):\n'
    
    p4='CUARTO: El empleador se compromete a remunerar al  Trabajador con un sueldo base  de '+ str(sueldo_base) + ' pesos '\
    'mensuales, en moneda nacional, más un 25% por concepto de Asignación de Estímulo mensual y Bono de Antigüedad de '+ str(bono_antiguedad)+'.\n'
  

    p5='TERCERA: Este anexo deja subsistente, en todo lo no modificado, el contrato individual de trabajo a que se alude en la '\
    'cláusula primera. El presente instrumento se firma en dos ejemplares de idéntico tenor, quedando uno en poder de cada '\
    'parte, dejándose expresa constancia de que el(la) Trabajador(a) ha recibido su copia al firmar el documento.\n\n\n\n' 

    documento=Document()
    
    documento.add_heading().alignment=WD_ALIGN_PARAGRAPH.CENTER
    documento.paragraphs[0].add_run('ANEXO \n\n').bold=True
    documento.paragraphs[0].alignment=WD_ALIGN_PARAGRAPH.CENTER
    documento.paragraphs[0].style='Normal'
    
    listaPs=[p1,p2,p3,p4,p5]

    for texto_parrafo in listaPs:
        parrafo=documento.add_paragraph()
        if texto_parrafo==p4:
            parrafo.add_run(texto_parrafo.decode('UTF-8')).italic = True
        else:
            parrafo.add_run(texto_parrafo.decode('UTF-8'))
        parrafo.alignment=WD_ALIGN_PARAGRAPH.JUSTIFY
        parrafo.style='Normal'
        
    
    
    documento.paragraphs[4].paragraph_format.left_indent = Pt(36)
    
    nombre_columnas = (str(nombre), '','pp. ASOCIACIÓN CHILENA DE SEGURIDAD')
    
    
    tabla = documento.add_table(rows=2, cols=len(nombre_columnas))


    
    for idx, name in enumerate(nombre_columnas):
        paragraph=tabla.rows[0].cells[idx].paragraphs[0]
        paragraph.alignment=WD_ALIGN_PARAGRAPH.CENTER
        
        if idx%2==0:
            run = paragraph.add_run('__________________________')
        
        paragraph = tabla.rows[1].cells[idx].paragraphs[0]
        paragraph.alignment=WD_ALIGN_PARAGRAPH.CENTER
        run = paragraph.add_run(name.decode('UTF-8'))
        run.bold = True


    documento.save('Anexo_'+RUT+'_'+nombre+'_'+'.docx')
    
